In [1]:
import os
import spotipy
import spotipy.util as util
import time
import json
import csv
import pandas as pd
import sys
import pickle

In [45]:
scope = 'user-library-read'
username = 'your_spotify_username'
token = util.prompt_for_user_token(username,scope,client_id=os.environ['SPOTIPY_CLIENT_ID'],client_secret=os.environ['SPOTIPY_CLIENT_SECRET'],redirect_uri=os.environ['SPOTIPY_REDIRECT_URI'])
print(token)
sp = spotipy.Spotify(auth=token)

BQD_aiATIzVDHSbQ35wavKiB9I6Zw5Bhx7MqQTIoIk5DNaQ7j1Ac5uxcGWB0z3oYVTi3FnsYgpzGKp2GaoYvLykhShwFpbWW_Gg4zPZMWeQnQtIc-sjZnJiCbSfubyco0lIjaCQ9oJbFI-zl0fYKExZXw12X4UU


In [3]:
# all_songs = {}  
# all_songs['songs'] = []

# i = 0
# for song in song_ids_list:
#     track = sp.track(song)
#     song_name = track['name']
#     artist = track['artists'][0]['name']
#     all_songs['songs'].append({"song_id": song, "song_name": song_name, "artist": artist})
#     if i % 100 == 0:
#         time.sleep(2)
#         print("i = ", i)
#     i+=1

In [4]:
# with open('song_id_and_names.json', 'w') as outfile:  
#     json.dump(all_songs, outfile)

In [5]:
with open('./all_track_id_features_0_7000.json') as f:
    all_track_id_features_0_7000 = json.load(f)
    
with open('./all_track_id_features_7000_14000.json') as f:
    all_track_id_features_7000_14000 = json.load(f)
    
with open('./all_track_id_features_14000_21000.json') as f:
    all_track_id_features_14000_21000 = json.load(f)
    
with open('./all_track_id_features_21000_35000.json') as f:
    all_track_id_features_21000_35000 = json.load(f)
    
with open('./all_track_id_features_35000_49000.json') as f:
    all_track_id_features_35000_49000 = json.load(f)
    
with open('./all_track_id_features_49000_63233.json') as f:
    all_track_id_features_49000_63233 = json.load(f)

In [6]:
# Dictionary mapping track_id to features
all_track_id_features_0_63233 = {**all_track_id_features_0_7000, **all_track_id_features_7000_14000, **all_track_id_features_14000_21000,\
                                **all_track_id_features_21000_35000, **all_track_id_features_35000_49000, **all_track_id_features_49000_63233}


In [7]:
print(len(all_track_id_features_0_63233.keys()))
# 63231 songs

63231


In [8]:
print(all_track_id_features_0_63233['0HzbDl381rdXtSOaJk20eH'])

[{'danceability': 0.509, 'energy': 0.904, 'key': 7, 'loudness': -3.686, 'mode': 1, 'speechiness': 0.0579, 'acousticness': 2.61e-05, 'instrumentalness': 0.716, 'liveness': 0.334, 'valence': 0.611, 'tempo': 141.949, 'type': 'audio_features', 'id': '0HzbDl381rdXtSOaJk20eH', 'uri': 'spotify:track:0HzbDl381rdXtSOaJk20eH', 'track_href': 'https://api.spotify.com/v1/tracks/0HzbDl381rdXtSOaJk20eH', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0HzbDl381rdXtSOaJk20eH', 'duration_ms': 187520, 'time_signature': 4}]


In [9]:
track_ids = all_track_id_features_0_63233.keys()
track_ids = list(track_ids)

In [49]:
all_songs = {}  
all_songs['songs'] = []
        
def get_song_metadata(sp, track_ids, start_index, end_index, block_size, all_songs, consecutive_errors):
    '''
    sp: Spotipy Object
    track_ids: List of song ids
    start_index, end_index: only consider track_ids[start_index : end_index]
    return: a list of the form [ (track_id, features, analysis), ... ]
    '''
    if consecutive_errors > 25:
        print("Too many consecutive errors.")
        filename = 'incomplete_song_id_and_names_0_' + str(end_index) + ".json"
        with open(filename, 'w') as outfile:  
            json.dump(all_songs, outfile)
        time.sleep(25)
        token = util.prompt_for_user_token(username,scope,client_id=os.environ['SPOTIPY_CLIENT_ID'],client_secret=os.environ['SPOTIPY_CLIENT_SECRET'],redirect_uri=os.environ['SPOTIPY_REDIRECT_URI'])
        print(token)
        sp = spotipy.Spotify(auth=token)
        print("new token")
        time.sleep(10)
        return get_song_metadata(sp, track_ids, start_index + 1, end_index, 100, all_songs, 0)
    try:
        for idx, track_id in enumerate(track_ids[start_index:end_index]):
            track = sp.track(track_id)
            song_name = track['name']
            artist = track['artists'][0]['name']
            all_songs['songs'].append({"song_id": track_id, "song_name": song_name, "artist": artist})
            
            # Every 100 iterations Sleep for 5 seconds to not overload API
            # Also save progress along the way to a .csv
            if idx % 100 == 0:
                print("SONG ID IS: ", track_id)
                print("Songs left: ", end_index - start_index)
                time.sleep(3)
                
            if (start_index) % 10000 == 0:
                filename = 'incomplete_song_id_and_names_0_' + str(start_index) + ".json"
                with open(filename, 'w') as outfile:  
                    json.dump(all_songs, outfile)
            start_index += 1
            consecutive_errors = 0
    except Exception as e:
        print(e)
        print("API called failed")
        print("Last song index was: ", start_index)
        print("Calling function again ")
        token = util.prompt_for_user_token(username,scope,client_id=os.environ['SPOTIPY_CLIENT_ID'],client_secret=os.environ['SPOTIPY_CLIENT_SECRET'],redirect_uri=os.environ['SPOTIPY_REDIRECT_URI'])
        print(token)
        sp = spotipy.Spotify(auth=token)
        #Call the function again from where we left off
        return get_song_metadata(sp, track_ids, start_index, end_index, 100, all_songs, 0)
    with open('song_id_and_names_0_63233.json', 'w') as outfile:
        json.dump(all_songs, outfile)
    return 'Success'

In [50]:
get_song_metadata(sp, track_ids, 0, 63231, 100, all_songs, 0)

SONG ID IS:  0HzbDl381rdXtSOaJk20eH
Songs left:  63231
SONG ID IS:  75Lv916cejvaYwggtOzuh8
Songs left:  63131
SONG ID IS:  7ahy3fhvPApYDTYtI8jHJN
Songs left:  63031
SONG ID IS:  51cqtusY2TwC6mbN8Vz8iA
Songs left:  62931
SONG ID IS:  09OrkXbEvQuPwQ39vPJcKo
Songs left:  62831
SONG ID IS:  0nfCqbh4NOXp1cBGb1jzif
Songs left:  62731
SONG ID IS:  6DJRq42LKg4I0z0BWFPQZA
Songs left:  62631
SONG ID IS:  08V7TKtRguQb7HiLpGcnzn
Songs left:  62531
SONG ID IS:  7iNIg7XDEaYECfWD5dJ9Va
Songs left:  62431
SONG ID IS:  1CAALs08dWg3BxFMaaS8qK
Songs left:  62331
SONG ID IS:  5YzckYO7G171MHg1UqkxJY
Songs left:  62231
SONG ID IS:  0B3CZlYaVRzTpAqfFoV4my
Songs left:  62131
SONG ID IS:  0Ay7pwWStfH00fpQlEaLFM
Songs left:  62031
SONG ID IS:  0dCYsUF1NXXyZRoY9lSM2q
Songs left:  61931
SONG ID IS:  2qMKlf1ZnbMUCiju8weGXV
Songs left:  61831
('Connection aborted.', OSError("(54, 'ECONNRESET')",))
API called failed
Last song index was:  1453
Calling function again 
BQAVVK_5rr-KRiAct9WnExlK0E5XVQ11m2Hh1k9xQq12RGxXJS

SONG ID IS:  3XgvfILdobj1b2pUa1czBl
Songs left:  50532
SONG ID IS:  6oBxyHMp4mwiRMbc52v76u
Songs left:  50432
SONG ID IS:  5zCuSaARd2hGfGTNf6o2Gq
Songs left:  50332
SONG ID IS:  6mfKEPTYiBAYZ9z0429jsp
Songs left:  50232
SONG ID IS:  6ZR6ZDHBsCOcLdshPANYTl
Songs left:  50132
SONG ID IS:  3YOvmznZ4YSokmTyBlVMJm
Songs left:  50032
SONG ID IS:  1mLUvpjtP8RAZIt9KoZwvv
Songs left:  49932
SONG ID IS:  7F3SaJ9nAhzldhzqfLPjxP
Songs left:  49832
SONG ID IS:  35p4xbdl7sLiBMo5iEmhkC
Songs left:  49732
SONG ID IS:  0Oujn60gIJx1JYKckV7W1h
Songs left:  49632
SONG ID IS:  2czUQBvuYvcXs5WY8r1ITC
Songs left:  49532
SONG ID IS:  5Hj24vdftqNsAO1UnCi8xJ
Songs left:  49432
SONG ID IS:  523qoxtSZJE2io945dAkVk
Songs left:  49332
SONG ID IS:  39tMDjMI14r0E8OnW3g8by
Songs left:  49232
SONG ID IS:  6Evl8zvEkPHHjRwLoh3dar
Songs left:  49132
SONG ID IS:  1tMFsas01GfuEUgtNYDpyo
Songs left:  49032
SONG ID IS:  23cj0rlc0UtTBaCg60VCkm
Songs left:  48932
SONG ID IS:  1iQuBIO2Lw3AXoWbnnRgea
Songs left:  48832
SONG ID IS

SONG ID IS:  0VKkyBKCXyR99navhVRqcP
Songs left:  36861
SONG ID IS:  7CqBFsnSUb2fKjl23EDYcN
Songs left:  36761
SONG ID IS:  5CmMQ6ATyBqdh48pIPSSWG
Songs left:  36661
SONG ID IS:  33xUiF1KYKR904pfKikQHD
Songs left:  36561
SONG ID IS:  0yzKJqFTXsbzWjUHaiOxD9
Songs left:  36461
SONG ID IS:  5EKD9tJnpY0YOVRNUFDT6R
Songs left:  36361
SONG ID IS:  6RzMzTdBOH05fQN8DvY2iQ
Songs left:  36261
SONG ID IS:  63lYhuhSqRToSdMo16elTf
Songs left:  36161
SONG ID IS:  1b2DXzL03cToFUhTwHX9lB
Songs left:  36061
SONG ID IS:  0dMQafMHQxzZIfBPUGWCQG
Songs left:  35961
SONG ID IS:  5E5XEWHRoV1sMDpQJCn2Qq
Songs left:  35861
SONG ID IS:  1dNIEtp7AY3oDAKCGg2XkH
Songs left:  35761
SONG ID IS:  2AXYCz4RlnuoCl7H5lasPm
Songs left:  35661
SONG ID IS:  5h36v6az1lmyzxM02bxwEi
Songs left:  35561
SONG ID IS:  4hi1oucIxMTNg83TT77FS8
Songs left:  35461
SONG ID IS:  4lerOTNr2tFWJCAmmhymhi
Songs left:  35361
SONG ID IS:  6IhFOgNP7YeQ3PoI8kcRLn
Songs left:  35261
SONG ID IS:  73wwn5fLy8E1pF5dGZs6Uf
Songs left:  35161
SONG ID IS

SONG ID IS:  38heLtyRuyWilWcGxFl8Sw
Songs left:  21961
SONG ID IS:  0fDY75Faj005D71yLPCKxX
Songs left:  21861
SONG ID IS:  5GXAXm5YOmYT0kL5jHvYBt
Songs left:  21761
SONG ID IS:  1XU8WnwGs6oqpt1qicvw34
Songs left:  21661
SONG ID IS:  6t2vcP38D08xYdioGyOUC6
Songs left:  21561
SONG ID IS:  2habSXqcJGExM6JJyskY7O
Songs left:  21461
SONG ID IS:  5hM5Lcvdw0C7Pf372DMbxG
Songs left:  21361
SONG ID IS:  2oSpQ7QtIKTNFfA08Cy0ku
Songs left:  21261
SONG ID IS:  1OmcAT5Y8eg5bUPv9qJT4R
Songs left:  21161
SONG ID IS:  6Q6dSJr0aBtTf6fRL5fMTo
Songs left:  21061
SONG ID IS:  1sVOxOEyBhqk16v3F3ffcg
Songs left:  20961
SONG ID IS:  7q5agbUOzZnVF4Yes3AD7I
Songs left:  20861
SONG ID IS:  4QUjn6cprNKiyA8dMTOQ33
Songs left:  20761
SONG ID IS:  1estw00E8DYSN92xu9beqa
Songs left:  20661
SONG ID IS:  3NcJu9876GBJuNU6vJOrbb
Songs left:  20561
SONG ID IS:  5Q58RkKyUafm15Syxg79DW
Songs left:  20461
SONG ID IS:  1gQDR2Jc7UhmYuwq1gJMd7
Songs left:  20361
SONG ID IS:  3aUFrxO1B8EW63QchEl3wX
Songs left:  20261
SONG ID IS

SONG ID IS:  4esEKIxsmfIJX7x6TWWyBM
Songs left:  7758
SONG ID IS:  4fMXQkXkfmO6KWKVwxvkWT
Songs left:  7658
SONG ID IS:  0EvphKjVl4hVGvvAFYoJt1
Songs left:  7558
SONG ID IS:  5JCzqqfUreBwpaVgoYqi7I
Songs left:  7458
SONG ID IS:  2NxAxyGVNihTzJ4exCrfh4
Songs left:  7358
SONG ID IS:  088OWVKqW0SPLbv7f78SUc
Songs left:  7258
SONG ID IS:  4G3EKvGtqcUSZqPQIWJFxd
Songs left:  7158
SONG ID IS:  1k5t0MC77UuwEp5FfZnKOd
Songs left:  7058
SONG ID IS:  44VyvABcrOHUXzEJVR33N6
Songs left:  6958
SONG ID IS:  6YJ6yrrt6Hw7TAFTLWqQxm
Songs left:  6858
SONG ID IS:  5ha8duNa729wTMDmz8aInL
Songs left:  6758
SONG ID IS:  7aPtEJs2zvQJTYKGjNvbfm
Songs left:  6658
SONG ID IS:  14FwKi5Lh5HRXeo6VNXlaN
Songs left:  6558
SONG ID IS:  6FugyKqhP4hZKofxRHoOV1
Songs left:  6458
SONG ID IS:  6z5Pd7UoLA6e1AlHYeVzqA
Songs left:  6358
SONG ID IS:  71GzM7SRPM36zKoiRyW5vm
Songs left:  6258
SONG ID IS:  15YJouNHJaoBqGTmdBleMX
Songs left:  6158
SONG ID IS:  0rPIMO7sLw5Z3FhSNLTA5Y
Songs left:  6058
SONG ID IS:  52mQhKYQNV0PDvK

'Success'

In [ ]:
print('wtf')

In [56]:
len(all_songs['songs'])
print(all_songs['songs'][2])

{'song_id': '1MMp1H2Kib2BCDtdL5nL63', 'song_name': 'Hey Jude', 'artist': 'Wilson Pickett'}


In [61]:
# Save everything to a CSV
null_songs = []
with open('all_song_metadata.csv','w') as fd:
    csv_writer = csv.writer(fd)
    csv_writer.writerow(["song_id", "song_name", "artist", "danceability", "energy", "key", "loudness", \
                        "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", \
                        "tempo", "duration_ms", "time_signature"])
    for song_dict in all_songs['songs']:
        song_id = song_dict['song_id']
        song_name = song_dict['song_name']
        song_artist = song_dict['artist']
        features = all_track_id_features_0_63233[song_id][0]
        if features is None:
            print(song_id)
            null_songs.append(song_id)
        else:
            csv_writer.writerow([song_id, song_name, song_artist, \
                                features["danceability"], features["energy"], features["key"],\
                                features["loudness"], features["mode"], features["speechiness"], \
                                features["acousticness"], features["instrumentalness"], features["liveness"], \
                                features["valence"], features["tempo"], features["duration_ms"], \
                                features["time_signature"] ])

3H9PkmNDnMCb1llejr4gyK
3knJwFwAEa1NjsvE8vMcAu
5cRkireIUGdPJjD3FF4jXZ
3MUkNrSJarCpxGMsE5nuDt
5pcsqeCdW32P4PfArtXIpo
0s5hSlD76edvVxoZE35nQh
5v5sNU2f67sbFXxmEtwdUW
5VPo4IXOerHSXbxfq1w4wL
2OMFMcSHeyCQtkXXwwtPVa
3hPmsIYWXlGdHJFuNUjvE7
2IDuUR3H01IQ8R6u3HDM2j
5AMigpxyqXNRS5cMEtR33t
1n30qkfDcobnWz2YRGGPdL
35Vz5sWeVVs0daA4og1UxR


In [63]:
# remove songs from database that have no features
print(len(all_track_id_features_0_63233.keys()))
for null_song_id in null_songs:
    del all_track_id_features_0_63233[null_song_id]
print(len(all_track_id_features_0_63233.keys()))

63231
63217


In [64]:
# Save all_track_id_features
with open('all_track_id_features_FINAL.json', 'w') as outfile:  
    json.dump(all_track_id_features_0_63233, outfile)

s
